<a href="https://colab.research.google.com/github/hiiiiiiiiiieve/SEN/blob/main/KoBERT_%EC%84%9C%EC%9A%B8%EA%B3%BC%ED%95%99%EA%B3%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **KoBERT**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 모듈 설치 및 실행

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292699 sha256=4c691d0355817eaadc01aec537782f6974b7de5dfae8550817acb8470c5dc49b
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp


In [ ]:
import gluonnlp as nlp

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.7 MB/s eta 0:00:00


In [ ]:
#Hugging face
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://github.com/SKTBrain/KoBERT.git

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-7ggfkw2y/kobert-tokenizer_3e01c996a3084d8a91b87ee6c493c9e6
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-7ggfkw2y/kobert-tokenizer_3e01c996a3084d8a91b87ee6c493c9e6
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=845d9c1e2a19dd60ec4e9c8e53448309699e2fbb857ee7848ad1407fe0ee32f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-7gsmtsx1/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-req-build-2hmxfk4_
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-req-build-2hmxfk4_
  Resolved https://github.com

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
!pip3 install kobert-transformers

In [ ]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 사용 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


# 데이터셋

In [ ]:
import pandas as pd

depressed_data = pd.read_csv("/content/drive/MyDrive/연구/dataset/wellness.csv", encoding='cp949')
depressed_data

,핵심증상,keyword(임상키워드),intent,특이사항,연관표현,utterance,utterance(2차),response(공감),임상질문그룹(연세의료원제공),utterance(긍정),utterance(부정),긍정에 대한 챗봇 답변,부정에 대한 챗봇 답변,추가발화(190917),추가발화 (191031)
0,지속되는우울한기분(우울감),우울,우울감,NaN,머리가 짓눌러지는 느낌/머리가 맑지 않다/침울하면서 잠도 못 자다/후회가 많고 침울...,우울해,임신해서 우울해,기분이 우울하시군요. 00님에게 스트레스 받는 일이 있었던 건 아닌지 걱정스러워요.,"거의 매일, 하루 중 대부분 시간을 우울한 기분으로 있거나 가라앉았던 적이 있나요?",응,아니,저런… (우는 이모티콘) 기분이 나아지지 않는 상태군요.,그렇군요. 기분이 계속 처지신다면 편안한 음악을 한번 들어보는 것도 좋을 것 같아요.,NaN,NaN
1,지속되는우울한기분(우울감),NaN,우울감,NaN,NaN,너무 우울해,아이 가지고 나서 우울해,마음이 우울해서 힘드셨군요. 00님의 소식이 궁금했는데 마침 저를 잘 찾아주셨네요.,NaN,어,아닌데?,힘든 상황이시군요. 저랑 더 얘기를 나눠볼까요?,그랬군요. 또 기분이 안 좋아지면 언제든 저에게 이야기해주세요.,NaN,NaN
2,지속되는우울한기분(우울감),NaN,우울감,NaN,NaN,우울해죽겠어,아이 가졌는데 기분 하나도 안 좋고 울적해,00님이 울적하시다니 저도 마음이 안 좋아요.,NaN,엉,노,한번 가라앉은 기분을 다시 돌리기는 참 어려운 것 같아요. 저와 계속 얘기해봐요.,그런 증상은 없으셨군요. 00님의 머릿속에 낀 먹구름이 얼른 걷혔으면 좋겠어요.,NaN,NaN
3,지속되는우울한기분(우울감),NaN,우울감,NaN,NaN,기분이 우울해,임신했는데 남편이 하나도 안 챙겨줘서 우울하다,얼굴에서 00님의 기분이 가라앉은 것이 느껴져요. 큰일이 있으신 건 아니었으면 좋겠어요.,NaN,예스,아니거든,NaN,NaN,NaN,NaN
4,지속되는우울한기분(우울감),NaN,우울감,NaN,NaN,우울모드임,진단 결과 안 좋게 나올 것 같아서 우울해,NaN,NaN,네,아니야,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19764,절망감,NaN,불안,NaN,NaN,NaN,그래도 잠못자고 불안한건 여전해요.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19765,절망감,NaN,불안,NaN,NaN,NaN,불안함에 항상 시달리니까 잠도 못잤어요.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19766,절망감,NaN,불안,NaN,NaN,NaN,불안하고 초조해서 잠이 안 와.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19767,절망감,NaN,불안,NaN,NaN,NaN,너무 불안하니까 밤만 되면 잠이 안 오고 너무 초조해.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#핵심증상 라벨 별 개수 알기
depressed_data["핵심증상"].value_counts()

지속되는우울한기분(우울감)      5305
초조함                 2697
절망감                 2294
수면                  1907
흥미나 즐거움의 감소(무기력)    1777
식욕-체중               1528
피로/활력상실             1245
자존감저하               1227
무가치함/죄책감            1224
사고력/집중력감소            565
Name: 핵심증상, dtype: int64

In [ ]:
#depressed_data 데이터 개수 확인
#참고 블로그(https://www.dinolabs.ai/271)에 비하여 1/3 크기

len(depressed_data)

19769

In [ ]:
#열이름 바꾸기
#utterance보다 utterance(2차)에 결측치 개수가 적기 때문에 후자 사용

#두 가지 열만 사용해보기

# depressed_data = depressed_data[['utterance(2차) ', 'response(공감)', '핵심증상']]
# depressed_data.columns = ['utterance', 'response', '감정']
# depressed_data.columns

depressed_data = depressed_data[['utterance(2차) ', '핵심증상']]
depressed_data.columns = ['utterance', '감정']
depressed_data.columns

Index(['utterance', '감정'], dtype='object')

In [ ]:
#답변열에 결측치가 많기 때문에 전처리 이전 결측치 데이터 개수 확인

depressed_data.dropna(inplace=True)
depressed_data['utterance'].isnull().sum()

<ipython-input-16-d1114fae04f1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depressed_data.dropna(inplace=True)


0

In [ ]:
#핵심증상 라벨 별 개수 알기
depressed_data['감정'].value_counts()

지속되는우울한기분(우울감)      5305
초조함                 2697
절망감                 2288
수면                  1867
흥미나 즐거움의 감소(무기력)    1777
식욕-체중               1528
피로/활력상실             1245
자존감저하               1227
무가치함/죄책감            1178
사고력/집중력감소            565
Name: 감정, dtype: int64

In [ ]:
# import pandas as pd

# # 원본 데이터프레임에서 감정에 해당하는 레이블을 갖는 데이터만 선택
# depressed_data_selected = depressed_data.loc[depressed_data['감정'].isin([
#     '지속되는우울한기분(우울감)', '초조함', '절망감', '수면', '흥미나 즐거움의 감소(무기력)',
#     '식욕-체중', '피로/활력상실', '자존감저하', '무가치함/죄책감', '사고력/집중력감소'
# ])]


# depressed_data_depression = depressed_data[depressed_data['감정'] == '지속되는우울한기분(우울감)']
# depressed_data_depression_sampled = depressed_data_depression.sample(n=3000, random_state=42)
# depressed_data_others = depressed_data[depressed_data['감정'] != '지속되는우울한기분(우울감)']
# balanced_data = pd.concat([depressed_data_others, depressed_data_depression_sampled], ignore_index=True)

# # 다시 이름 바꾸기
# depressed_data = balanced_data

# # 결과 확인
# print(depressed_data['감정'].value_counts())

In [ ]:
#depressed_data 데이터 개수 확인
#참고 블로그(https://www.dinolabs.ai/271)에 비하여 1/3 크기

len(depressed_data)

19677

In [ ]:
#감정 클래스를 숫자로 변경하는 전처리

depressed_data.loc[(depressed_data['감정'] == "지속되는우울한기분(우울감)"), '감정'] = 0
depressed_data.loc[(depressed_data['감정'] == "흥미나 즐거움의 감소(무기력)"), '감정'] = 1
depressed_data.loc[(depressed_data['감정'] == "식욕-체중"), '감정'] = 2
depressed_data.loc[(depressed_data['감정'] == "수면"), '감정'] = 3
depressed_data.loc[(depressed_data['감정'] == "초조함"), '감정'] = 4
depressed_data.loc[(depressed_data['감정'] == "피로/활력상실"), '감정'] = 5
depressed_data.loc[(depressed_data['감정'] == "무가치함/죄책감"), '감정'] = 6
depressed_data.loc[(depressed_data['감정'] == "자존감저하"), '감정'] = 6
depressed_data.loc[(depressed_data['감정'] == "사고력/집중력감소"), '감정'] = 7
depressed_data.loc[(depressed_data['감정'] == "절망감"), '감정'] = 8

#답변과 감정만 추출하여 리스트 생성

data_list = []
for ques, label in zip(depressed_data['utterance'], depressed_data['감정'])  :
    depressed_data = []
    depressed_data.append(ques)
    depressed_data.append(str(label))

    data_list.append(depressed_data)

data_list

<ipython-input-20-c6d5e730c0f6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depressed_data.loc[(depressed_data['감정'] == "지속되는우울한기분(우울감)"), '감정'] = 0
<ipython-input-20-c6d5e730c0f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depressed_data.loc[(depressed_data['감정'] == "흥미나 즐거움의 감소(무기력)"), '감정'] = 1
<ipython-input-20-c6d5e730c0f6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depressed_data.loc[(depressed_data['감정'] == "식욕-

[['임신해서 우울해', '0'],
 ['아이 가지고 나서 우울해', '0'],
 ['아이 가졌는데 기분 하나도 안 좋고 울적해', '0'],
 ['임신했는데 남편이 하나도 안 챙겨줘서 우울하다', '0'],
 ['진단 결과 안 좋게 나올 것 같아서 우울해 ', '0'],
 ['먹고 싶은 거 못 먹어서 우울해', '0'],
 ['하고 싶은 거 못 해서 이렇게 우울할 수가 없어', '0'],
 ['술을 못 마시면 이렇게 우울해지는 구나', '0'],
 ['술 못 마시니까 진짜 우울하다', '0'],
 ['남편이 안 챙겨주니까 우울해지는 것 같아', '0'],
 ['남편이 안 챙겨줘서 진심 우울해요', '0'],
 ['살이 안 빠져서 우울해', '0'],
 ['노산이라 우울해요', '0'],
 ['대화가 안 통해서 우울한 것 같아', '0'],
 ['경력이 단절된다는 생각만 하면 우울한 것 같아요', '0'],
 ['대화가 안 통하니까 우울하잖아', '0'],
 ['오늘 상담 받고 와서 우울해요', '0'],
 ['유도 실패해서 개 우울해', '0'],
 ['재검 해야 한다고 해서 기분이 안 좋아', '0'],
 ['원인은 모르겠는데 자꾸 꿀꿀해', '0'],
 ['기분이 계속 가라앉아서 땅 속에 꺼진 느낌이야', '0'],
 ['별 일 없는데도 우울한 기분이 계속 돼', '0'],
 ['대체적으로 매일매일 우울해', '0'],
 ['밤에 특히 더 우울하다', '0'],
 ['우울하니까 물건도 집어 던져', '0'],
 ['우울한 상태에서 나도 모르게 화를 내버려', '0'],
 ['계속 우울해서 술 마셨어', '0'],
 ['왜 그런지 맨날 울어', '0'],
 ['혼자 있으면 더 우울해', '0'],
 ['혼자가 되면 우울해지는 것 같아', '0'],
 ['우울해서 담배 찾아', '0'],
 ['우울하면서 머리가 짓눌러지는 것 같은 느낌이 들어', '0'],
 ['혼자 있으면 꿀꿀해', '0'],
 ['오빠가 직장에 있으면 꿀꿀해지는 것 같아', '0'],
 

# **KoBERT**

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64 #loss 줄이기 위해 높여보기
warmup_ratio = 0.1
num_epochs = 3 #5로 할 경우 런타임이 끊겨서 3으로 줄여봄
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5 #loss 줄이기 위해 낮춰보기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len, pad=True, pair=False):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair
        self.dataset = dataset

    def __getitem__(self, idx):
        if self.pair:
            text = self.dataset[idx][0][0]
            text_pair = self.dataset[idx][0][1]
        else:
            text = self.dataset[idx][0]

        inputs = self.tokenizer.encode_plus(
            text,
            text_pair if self.pair else None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=self.pad,
            truncation=False
        )

        item = {
            'input_ids': torch.tensor(inputs['input_ids']),
            'attention_mask': torch.tensor(inputs['attention_mask']),
            'token_type_ids': torch.tensor(inputs['token_type_ids'])
        }

        item['labels'] = int(self.dataset[idx][1])

        # 패딩된 시퀀스/길이와 타입에 대한 내용/어텐션 마스크 시퀀스 세 가지 배열을 얻을 수 있도록 함
        padded_seq = torch.tensor(inputs['input_ids']).numpy()
        len_and_type = list(torch.tensor(inputs['token_type_ids']).shape) #dtype은 출력 안됨
        attention_mask_seq = torch.tensor(inputs['attention_mask']).numpy()

        return np.array(padded_seq), np.array(len_and_type), np.array(attention_mask_seq), item['labels']


    def __len__(self):
        return len(self.dataset)


# 데이터셋 생성
train_dataset = BERTDataset(dataset_train, tokenizer, max_len, True, True)
test_dataset = BERTDataset(dataset_test, tokenizer, max_len, True, True)

# 데이터로더 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_dataset[0]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(array([   2, 1185,    3,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1]),
 array([64]),
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 1)

#KoBERT 모델

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=10,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

#Train

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_loader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-31-dcae0e76ddc1>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):


  0%|          | 0/492 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.359971046447754 train acc 0.0625
epoch 1 batch id 201 loss 2.206393003463745 train acc 0.23911691542288557
epoch 1 batch id 401 loss 1.9074313640594482 train acc 0.265430174563591
epoch 1 train acc 0.27345283151107375


<ipython-input-31-dcae0e76ddc1>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_loader)):


  0%|          | 0/123 [00:00<?, ?it/s]

...

epoch 2 test acc 0.3630589430894309


  0%|          | 0/492 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 1.61808443069458 train acc 0.4375
epoch 3 batch id 201 loss 1.710221529006958 train acc 0.3793532338308458
epoch 3 batch id 401 loss 1.9130918979644775 train acc 0.38598815461346636
epoch 3 train acc 0.3896613050182226


  0%|          | 0/123 [00:00<?, ?it/s]

epoch 3 test acc 0.38846544715447157


In [ ]:
import pickle

# 결과를 저장할 딕셔너리 생성
result = {
    "train_history": train_history,
    "test_history": test_history,
    "loss_history": loss_history
}

# 결과를 파일로 저장
with open("result.pkl", "wb") as f:
    pickle.dump(result, f)
print("결과가 저장되었습니다.")

결과가 저장되었습니다.


In [ ]:
# 아래 결합한 부분에서 코드 실행

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, tokenizer, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("우울감이")
            elif np.argmax(logits) == 1:
                test_eval.append("흥미 또는 즐거움의 상실이")
            elif np.argmax(logits) == 2:
                test_eval.append("식욕 또는 체중 변화가")
            elif np.argmax(logits) == 3:
                test_eval.append("수면장애가")
            elif np.argmax(logits) == 4:
                test_eval.append("정신운동 지체 또는 초조함이")
            elif np.argmax(logits) == 5:
                test_eval.append("피로 또는 활력 저하가")
            elif np.argmax(logits) == 6:
                test_eval.append("무가치함 또는 죄책감이")
            elif np.argmax(logits) == 7:
                test_eval.append("사고력 및 집중력 감소가")
            elif np.argmax(logits) == 8:
                test_eval.append("자살에 대한 생각이")


        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
# 아래 결합한 부분에서 코드 실행

end = 1
while end == 1 :
    sentence = input("현재 어떤 기분 혹은 상태가 느껴지시나요? : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

현재 어떤 기분 혹은 상태가 느껴지시나요? : 너무 우울해
>> 입력하신 내용에서 우울감이 느껴집니다.


현재 어떤 기분 혹은 상태가 느껴지시나요? : 하는 일이 즐겁지 않아
>> 입력하신 내용에서 흥미 또는 즐거움의 상실이 느껴집니다.


현재 어떤 기분 혹은 상태가 느껴지시나요? : 입맛이 없어
>> 입력하신 내용에서 식욕 또는 체중 변화가 느껴집니다.


현재 어떤 기분 혹은 상태가 느껴지시나요? : 하루종일 잠만 자
>> 입력하신 내용에서 수면장애가 느껴집니다.


현재 어떤 기분 혹은 상태가 느껴지시나요? : 뭘 어떻게 해야 할지 모르겠어
>> 입력하신 내용에서 우울감이 느껴집니다.


현재 어떤 기분 혹은 상태가 느껴지시나요? : 이유 없이 피로감이 쌓여
>> 입력하신 내용에서 우울감이 느껴집니다.


현재 어떤 기분 혹은 상태가 느껴지시나요? : 작은 일에도 미안함이 크게 느껴져
>> 입력하신 내용에서 우울감이 느껴집니다.


현재 어떤 기분 혹은 상태가 느껴지시나요? : 내 일에 집중이 잘 안돼
>> 입력하신 내용에서 우울감이 느껴집니다.


현재 어떤 기분 혹은 상태가 느껴지시나요? : 죽고 싶어
>> 입력하신 내용에서 자살에 대한 생각이 느껴집니다.


